In [1]:
import mesa
print(mesa.__version__)

3.1.1


In [2]:
# Data visualization tools.
import seaborn as sns

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

#Visualization
from mesa.visualization import SolaraViz, make_plot_component, make_space_component

#For Dice control
import random

<IPython.core.display.Javascript object>

In [3]:
#Dictionary
snakes = {16: 6, 47: 26, 49: 11, 62: 19, 64: 60, 87: 24, 93: 73, 95: 75, 99: 78}
ladders = {2: 38, 7: 14, 8: 31, 15: 26, 28: 84, 36: 44, 51: 67, 71: 91, 78: 98, 87: 94}

In [4]:
class PlayerAgent(mesa.Agent):
    def __init__(self, model):
        super().__init__(model)
        self.position = 1  # Start at position 1 (bottom-left of the board)
        self.grid_pos = (0, 0)  # Start at (0, 0)

    def roll_dice(self):
        """Roll a dice and return a number between 1 and 6."""
        return random.randint(1, 6)

    def calculate_new_position(self, new_position):
        """Convert board position to grid coordinates."""
        row = (new_position - 1) // 10
        col = (new_position - 1) % 10
        if row % 2 == 1:  # Reverse direction for odd rows
            col = 9 - col
        return (col, row)

    def move(self):
        """Move the player based on dice roll and handle snakes/ladders."""
        dice_roll = self.roll_dice()
        new_position = self.position + dice_roll
        if new_position > 100:
            new_position = self.position  # Stay in the same position if roll exceeds 100

        # Check for snakes or ladders
        if new_position in self.model.snakes:
            print(f"Player {self.unique_id} encountered a snake at {new_position}! Sliding down...")
            new_position = self.model.snakes[new_position]
        elif new_position in self.model.ladders:
            print(f"Player {self.unique_id} climbed a ladder at {new_position}!")
            new_position = self.model.ladders[new_position]

        # Update position and grid
        self.position = new_position
        self.grid_pos = self.calculate_new_position(new_position)

        # Move agent in the grid
        self.model.grid.move_agent(self, self.grid_pos)
        print(f"Player {self.unique_id} rolled {dice_roll} and moved to position {self.position} ({self.grid_pos})")

    def step(self):
        """Define the player's actions for each step."""
        self.move()
        if self.position == 100:
            print(f"Player {self.unique_id} wins!")
            self.model.winner = self.unique_id
            self.model.running = False  # End the simulation

In [5]:
class SnakeAndLaddersModel(mesa.Model):
    def __init__(self, num_players):
        super().__init__()
        self.num_players = num_players
        self.grid = mesa.space.MultiGrid(10, 10, torus=False)  # 10x10 board
        self.snakes = {16: 6, 47: 26, 49: 11, 62: 19, 64: 60, 87: 24, 93: 73, 95: 75, 99: 78}
        self.ladders = {2: 38, 7: 14, 8: 31, 15: 26, 28: 84, 36: 44, 51: 67, 71: 91, 78: 98, 87: 94}
        self.winner = None

        self.player_agents = []  # Custom storage, not overwriting model.agents

        # Add players to the model
        for _ in range(self.num_players):
            player = PlayerAgent(self)
            self.player_agents.append(player)
            self.grid.place_agent(player, (0, 0))  # Place agent at the start
            print(f"Player {player.unique_id} created")  # Verify unique IDs

    def step(self):
        """Advance the model by one step."""
        random.shuffle(self.player_agents)  # Shuffle players manually
        for agent in self.player_agents:
            agent.step()

In [6]:
# Create initial model instance with 2 players
model1 = SnakeAndLaddersModel(num_players=2)

# Run the model for a few steps to see the output
print("Starting Simulation...\n")
for i in range(5):  # Run 5 steps
    print(f"\n--- Step {i+1} ---")
    model1.step()

print("\nSimulation Complete.")

Player 1 created
Player 2 created
Starting Simulation...


--- Step 1 ---
Player 1 climbed a ladder at 2!
Player 1 rolled 1 and moved to position 38 ((2, 3))
Player 2 rolled 2 and moved to position 3 ((2, 0))

--- Step 2 ---
Player 2 rolled 3 and moved to position 6 ((5, 0))
Player 1 rolled 4 and moved to position 42 ((1, 4))

--- Step 3 ---
Player 1 rolled 6 and moved to position 48 ((7, 4))
Player 2 rolled 5 and moved to position 11 ((9, 1))

--- Step 4 ---
Player 1 climbed a ladder at 51!
Player 1 rolled 3 and moved to position 67 ((6, 6))
Player 2 encountered a snake at 16! Sliding down...
Player 2 rolled 5 and moved to position 6 ((5, 0))

--- Step 5 ---
Player 1 rolled 5 and moved to position 72 ((8, 7))
Player 2 rolled 4 and moved to position 10 ((9, 0))

Simulation Complete.


In [7]:
def agent_portrayal(agent):
    """Defines how agents are visualized on the grid."""
    if isinstance(agent, PlayerAgent):
        return {

        }

In [8]:
model_params = {
    "num_players": 2  # Default number of players; can be modified in the visualization
}

In [9]:
# Create initial model instance
model1 = SnakeAndLaddersModel(num_players=2) #keyword arguments

SpaceGraph = make_space_component(agent_portrayal)

Player 1 created
Player 2 created


In [10]:
page = SolaraViz(
    model1,
    components=[SpaceGraph],
    model_params=model_params,
    name="Snake N Ladders Model",
)
# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text